In [13]:
import yfinance as yf
import pandas as pd
import time
from pathlib import Path

TICKERS = {
    "large": {
        "banking": ["UBSG.SW"],
        "insurance": ["ZURN.SW"],
        "pharma": ["ROG.SW", "NOVN.SW", "SANN.SW"],
        "food_retail": ["NESN.SW", "EMMN.SW", "ARYN.SW"],
        "insurance": ["SREN.SW"],               # Swiss Re
        "pharma": ["LONN.SW", "ALC.SW"],        # Lonza, Alcon
        "food_retail": ["GIVN.SW"],             # Givaudan
        "consumer": ["CFR.SW"],                 # Richemont
        "tech_industrial": ["ABBN.SW", "HOLN.SW", "SCMN.SW"],  # ABB, Holcim, Swisscom
    },
    "mid": {
        "insurance": ["HELN.SW", "SLHN.SW", "SQN.SW"],
        "energy": ["BKW.SW", "REHN.SW"],
        "tech": ["LOGN.SW", "UHR.SW", "LAND.SW"],
    },
    "small": {
        "banking": ["VONN.SW", "BAER.SW", "BLKB.SW"],
        "insurance": ["BALN.SW"],  # Mobiliar unlisted; add others as needed
    },
}

def flatten(d):
    rows = []
    for size, sectors in d.items():
        for sector, syms in sectors.items():
            for s in syms:
                rows.append({"size": size, "sector": sector, "ticker": s})
    return pd.DataFrame(rows)

def validate_tickers(tickers):
    ok, bad = [], []
    for t in tickers:
        try:
            info = yf.Ticker(t).fast_info
            # fast_info usually has fields when valid; empty dict for bad/OTC
            if info and getattr(info, "last_price", None) is not None:
                ok.append(t)
            else:
                bad.append(t)
        except Exception:
            bad.append(t)
    return ok, bad

def download_batch(tickers, start="2018-01-01", end="2024-12-31", out_dir="data/stock_prices", max_retry=3):
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    manifest = []

    # Batch download (fewer HTTP calls). Keep auto_adjust for total-return style close.
    remaining = set(tickers)
    attempt = 0
    while remaining and attempt < max_retry:
        attempt += 1
        try:
            data = yf.download(
                tickers=list(remaining),
                start=start, end=end,
                auto_adjust=True, group_by="ticker", threads=True, progress=False
            )
        except Exception:
            data = None

        fetched = set()
        if data is not None and not data.empty:
            # When multiple: columns become (ticker, field)
            for t in list(remaining):
                try:
                    df = data[t].dropna(how="all")
                    if not df.empty:
                        df.to_csv(f"{out_dir}/{t}.csv")
                        manifest.append({"ticker": t, "status": "ok", "rows": len(df)})
                        fetched.add(t)
                    else:
                        manifest.append({"ticker": t, "status": "empty", "rows": 0})
                        fetched.add(t)
                except Exception:
                    # Single-ticker case or structure mismatch: try direct download
                    try:
                        df2 = yf.download(t, start=start, end=end, auto_adjust=True, progress=False)
                        if not df2.empty:
                            df2.to_csv(f"{out_dir}/{t}.csv")
                            manifest.append({"ticker": t, "status": "ok_solo", "rows": len(df2)})
                            fetched.add(t)
                    except Exception:
                        pass

        remaining -= fetched
        if remaining:
            time.sleep(1.0 * attempt)  # backoff

    # Anything still remaining → mark failed
    for t in remaining:
        manifest.append({"ticker": t, "status": "failed", "rows": 0})

    pd.DataFrame(manifest).to_csv(f"{out_dir}/_manifest.csv", index=False)
    return pd.DataFrame(manifest)

# ---- Run it ----
universe = flatten(TICKERS)
ok_syms, bad_syms = validate_tickers(universe["ticker"].tolist())
print("Valid symbols:", ok_syms)
print("Problem symbols:", bad_syms)

manifest = download_batch(ok_syms, start="2018-01-01", end="2024-12-31", out_dir="data/stock_prices")
print(manifest.value_counts(["status"]))


Valid symbols: ['UBSG.SW', 'SREN.SW', 'LONN.SW', 'ALC.SW', 'GIVN.SW', 'CFR.SW', 'ABBN.SW', 'HOLN.SW', 'SCMN.SW', 'HELN.SW', 'SLHN.SW', 'SQN.SW', 'BKW.SW', 'REHN.SW', 'LOGN.SW', 'UHR.SW', 'LAND.SW', 'VONN.SW', 'BAER.SW', 'BLKB.SW', 'BALN.SW']
Problem symbols: []
status
ok        21
Name: count, dtype: int64


In [14]:
from pathlib import Path
import pandas as pd

for p in Path("data/stock_prices").glob("*.csv"):
    if p.name.startswith("_"): 
        continue
    head = pd.read_csv(p, nrows=5)
    print(p.name, "→ columns:", list(head.columns))


ABBN.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
ALC.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
ALPN.SW.csv → columns: ['Price', 'Close', 'High', 'Low', 'Open', 'Volume']
ARYN.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
BAER.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
BALN.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
BKW.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
BLKB.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
CFR.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
EMMN.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
GIVN.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
HELN.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
HOLN.SW.csv → columns: ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
LAND.SW.csv → columns: ['Date', 'Open', 

In [15]:
import pandas as pd
from pathlib import Path
import numpy as np

VALID = {
    "UBSG.SW","ZURN.SW","ROG.SW","NOVN.SW","SANN.SW",
    "NESN.SW","EMMN.SW","ARYN.SW",
    "HELN.SW","SLHN.SW","SQN.SW",
    "BKW.SW","REHN.SW",
    "LOGN.SW","UHR.SW","LAND.SW",
    "VONN.SW","BAER.SW","BLKB.SW","BALN.SW",
    "SREN.SW","CFR.SW","LONN.SW","HOLN.SW",
    "SCMN.SW","GIVN.SW","ALC.SW","ABBN.SW"
}



RAW_DIR = Path("data/stock_prices")
CLEAN_DIR = Path("data/stock_prices_clean")
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

def normalize_csv(p: Path):
    # skip non-universe files
    ticker = p.stem
    if ticker not in VALID:
        print(f"Skipping (not in universe): {p.name}")
        return None

    try:
        df = pd.read_csv(p)
    except Exception as e:
        print(f"Read fail {p.name}: {e}")
        return None

    if df.empty:
        print(f"Empty file: {p.name}")
        return None

    cols = list(df.columns)

    # Fix missing Date (some files have 'Price' as the first column)
    if "Date" not in cols:
        if "Price" in cols:
            df = df.rename(columns={"Price": "Date"})
        elif "Unnamed: 0" in cols:
            df = df.rename(columns={"Unnamed: 0": "Date"})
        else:
            print(f"No Date-like column in {p.name}, skipping.")
            return None

    # Parse dates
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])

    # Ensure required OHLCV columns exist
    needed = {"Open","High","Low","Close","Volume"}
    missing = needed - set(df.columns)
    if missing:
        print(f"Missing {missing} in {p.name}, skipping.")
        return None

    # Build normalized DF
    out = df[["Date","Open","High","Low","Close","Volume"]].copy()

    # If Adj Close missing, use Close as a fallback (not total return)
    if "Adj Close" in df.columns:
        out["Adj Close"] = df["Adj Close"]
    else:
        out["Adj Close"] = df["Close"]

    out = (out.rename(columns=str.lower)
              .sort_values("date")
              .assign(ticker=ticker))

    out.to_csv(CLEAN_DIR / p.name, index=False)
    return out

# Run normalization
normalized = []
for p in RAW_DIR.glob("*.csv"):
    if p.name.startswith("_"):
        continue
    res = normalize_csv(p)
    if res is not None:
        normalized.append(res)

if not normalized:
    raise RuntimeError("No normalized files produced.")

print(f"Normalized {len(normalized)} files → {CLEAN_DIR}")

# ---------- Rebuild the panel using the CLEAN_DIR ----------
def load_prices_clean(data_dir=CLEAN_DIR):
    data=[]
    for p in Path(data_dir).glob("*.csv"):
        df = pd.read_csv(p, parse_dates=["date"])
        if df.empty: 
            continue
        data.append(df[["date","open","high","low","close","adj close","volume","ticker"]]
                      .rename(columns={"adj close":"adj_close"}))
    return pd.concat(data, ignore_index=True)

px = load_prices_clean()
print(px.groupby("ticker")["date"].agg(['min','max','count']).sort_index().tail())


Skipping (not in universe): ALPN.SW.csv
Skipping (not in universe): SWON.SW.csv
Normalized 28 files → data\stock_prices_clean
               min        max  count
ticker                              
SREN.SW 2018-01-03 2024-12-30   1759
UBSG.SW 2018-01-03 2024-12-30   1759
UHR.SW  2018-01-03 2024-12-30   1759
VONN.SW 2018-01-03 2024-12-30   1759
ZURN.SW 2018-01-03 2024-12-30   1759


In [16]:
import pandas as pd
from pathlib import Path
import numpy as np

def load_prices_clean(data_dir="data/stock_prices_clean"):
    data=[]
    for p in Path(data_dir).glob("*.csv"):
        df = pd.read_csv(p, parse_dates=["date"])
        if df.empty: 
            continue
        df = df.rename(columns={"adj close":"adj_close"})
        data.append(df[["date","open","high","low","close","adj_close","volume","ticker"]])
    return pd.concat(data, ignore_index=True)

panel = load_prices_clean()
print(panel.head())
print(panel["ticker"].nunique(), "tickers loaded")


        date       open       high        low      close  adj_close  \
0 2018-01-03  19.562587  19.749111  19.547665  19.719267  19.719267   
1 2018-01-04  19.838642  19.958017  19.749111  19.935635  19.935635   
2 2018-01-05  19.920712  20.084852  19.883406  20.084852  20.084852   
3 2018-01-08  20.099775  20.137079  19.920713  20.092314  20.092314   
4 2018-01-09  19.972938  20.196766  19.935634  20.196766  20.196766   

      volume   ticker  
0  5519259.0  ABBN.SW  
1  5738092.0  ABBN.SW  
2  4435594.0  ABBN.SW  
3  5029780.0  ABBN.SW  
4  6974533.0  ABBN.SW  
28 tickers loaded


In [17]:
import pandas as pd
import numpy as np
from pathlib import Path

# ---- 1) Reload cleaned prices ----
def load_prices_clean(data_dir="data/stock_prices_clean"):
    data=[]
    for p in Path(data_dir).glob("*.csv"):
        df = pd.read_csv(p, parse_dates=["date"])
        if df.empty: 
            continue
        df = df.rename(columns={"adj close":"adj_close"})
        data.append(df[["date","open","high","low","close","adj_close","volume","ticker"]])
    return pd.concat(data, ignore_index=True)

panel = load_prices_clean()

# ---- 2) Add basic features (so QC has ret_1d, etc.) ----
def add_features(g):
    g = g.sort_values("date").copy()
    g["ret_1d"] = g["adj_close"].pct_change()
    g["logret_1d"] = np.log(g["adj_close"]).diff()
    return g

panel = panel.groupby("ticker", group_keys=False).apply(add_features).dropna().reset_index(drop=True)

print("✅ Reloaded panel with features:", panel.shape)

# ---- 3) QC checks ----
# a) Missing data ratios
miss = (panel
        .groupby("ticker")["adj_close"]
        .apply(lambda s: s.isna().mean())
        .sort_values(ascending=False))
print("\nMissing ratio per ticker (0 = perfect):\n", miss)

# b) Trading-day alignment
non_mono = panel.groupby("ticker")["date"].apply(lambda s: not s.is_monotonic_increasing)
print("\nAny non-monotonic sequences?", bool(non_mono.any()))

ref_dates = set(panel.loc[panel.ticker.eq("UBSG.SW"), "date"])
gap_counts = {t: len(ref_dates - set(g["date"])) for t, g in panel.groupby("ticker")}
print("\nGaps vs UBSG.SW calendar (top 10):\n", pd.Series(gap_counts).sort_values(ascending=False).head(10))

# c) Return outliers
q = panel.groupby("ticker")["ret_1d"].quantile([0.001,0.01,0.99,0.999]).unstack()
print("\nExtreme return quantiles:\n", q)

abs_out = (panel.assign(abs_ret=lambda d: d["ret_1d"].abs())
           .sort_values("abs_ret", ascending=False)
           .loc[:,["date","ticker","ret_1d"]]
           .head(15))
print("\nTop absolute daily moves:\n", abs_out.to_string(index=False))


✅ Reloaded panel with features: (48908, 10)

Missing ratio per ticker (0 = perfect):
 ticker
ABBN.SW    0.0
ALC.SW     0.0
ARYN.SW    0.0
BAER.SW    0.0
BALN.SW    0.0
BKW.SW     0.0
BLKB.SW    0.0
CFR.SW     0.0
EMMN.SW    0.0
GIVN.SW    0.0
HELN.SW    0.0
HOLN.SW    0.0
LAND.SW    0.0
LOGN.SW    0.0
LONN.SW    0.0
NESN.SW    0.0
NOVN.SW    0.0
REHN.SW    0.0
ROG.SW     0.0
SANN.SW    0.0
SCMN.SW    0.0
SLHN.SW    0.0
SQN.SW     0.0
SREN.SW    0.0
UBSG.SW    0.0
UHR.SW     0.0
VONN.SW    0.0
ZURN.SW    0.0
Name: adj_close, dtype: float64

Any non-monotonic sequences? False

Gaps vs UBSG.SW calendar (top 10):
 ALC.SW     316
ABBN.SW      0
ARYN.SW      0
BAER.SW      0
BALN.SW      0
BKW.SW       0
BLKB.SW      0
CFR.SW       0
EMMN.SW      0
GIVN.SW      0
dtype: int64

Extreme return quantiles:
             0.001     0.010     0.990     0.999
ticker                                         
ABBN.SW -0.076706 -0.043446  0.040719  0.061835
ALC.SW  -0.076492 -0.042929  0.051251  0.129585

C:\Users\mkcak\AppData\Local\Temp\ipykernel_30120\2569322819.py:25: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  panel = panel.groupby("ticker", group_keys=False).apply(add_features).dropna().reset_index(drop=True)


In [18]:
import pandas as pd
import numpy as np

# --- 0) START from your current `panel` (already loaded and has ret_1d/logret_1d) ---

# A) (Optional) Align to a common trading calendar (drop days missing for reference ticker)
#    This avoids forward-filling prices across non-trading days for thin names.
REF = "UBSG.SW"
ref_dates = set(panel.loc[panel.ticker.eq(REF), "date"])
panel_aligned = panel.loc[panel["date"].isin(ref_dates)].copy()

# Quick check
cal_gaps = (panel_aligned.groupby("ticker")["date"]
            .apply(lambda s: len(ref_dates - set(s))))
print("Calendar gaps (should be 0 after alignment):\n", cal_gaps.sort_values(ascending=False).head())

# B) Winsorize daily returns per ticker (robust to biotech-style spikes)
#    We'll cap at the 0.5% / 99.5% quantiles PER TICKER.
def winsorize_group(g, col="ret_1d", lower_q=0.005, upper_q=0.995):
    g = g.sort_values("date").copy()
    lo, hi = g[col].quantile([lower_q, upper_q]).values
    g[col + "_w"] = g[col].clip(lo, hi)
    # Recompute log returns consistently from winsorized simple returns
    g["logret_1d_w"] = np.log1p(g[col + "_w"])
    return g

# Use the more explicit include_groups=False to silence future warning
panel_w = (panel_aligned
           .groupby("ticker", group_keys=False, sort=False)
           .apply(winsorize_group, include_groups=False))

# C) Recompute rolling features based on winsorized log returns (leakage-safe)
def add_roll_feats(g):
    g = g.sort_values("date").copy()
    # Annualized realized vol on winsorized log returns
    for w in (5, 21, 63):
        g[f"rv_{w}d_w"] = g["logret_1d_w"].rolling(w).std() * np.sqrt(252)
        g[f"ma_{w}d"] = g["adj_close"].rolling(w).mean()
        g[f"mom_{w}d"] = g["adj_close"].pct_change(w)
    # Keep existing targets if you have them; if not, you can (re


Calendar gaps (should be 0 after alignment):
 ticker
ALC.SW     316
ABBN.SW      0
ARYN.SW      0
BAER.SW      0
BALN.SW      0
Name: date, dtype: int64


In [19]:
panel_w = (panel_aligned
           .groupby("ticker", group_keys=False, sort=False)
           .apply(winsorize_group, include_groups=False))



In [20]:
print(panel_w.columns)


Index(['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'ret_1d',
       'logret_1d', 'ret_1d_w', 'logret_1d_w'],
      dtype='object')


In [21]:
import pandas as pd
import numpy as np

# --- assume you already have `panel` with columns incl. 'ticker' ---
# If not, reload from clean CSVs:
# from pathlib import Path
# def load_prices_clean(data_dir="data/stock_prices_clean"):
#     data=[]
#     for p in Path(data_dir).glob("*.csv"):
#         df = pd.read_csv(p, parse_dates=["date"]).rename(columns={"adj close":"adj_close"})
#         data.append(df[["date","open","high","low","close","adj_close","volume","ticker"]])
#     return pd.concat(data, ignore_index=True)
# panel = load_prices_clean()

# 1) Align to UBSG calendar (as before)
REF = "UBSG.SW"
ref_dates = set(panel.loc[panel.ticker.eq(REF), "date"])
panel_aligned = panel.loc[panel["date"].isin(ref_dates)].copy()

# 2) Winsorize per-ticker and KEEP ticker via group name
def winsorize_group(g, col="ret_1d", lower_q=0.005, upper_q=0.995):
    t = g.name  # <-- group/ticker name
    g = g.sort_values("date").copy()
    # ensure ticker present
    g["ticker"] = t
    # if ret_1d/logret_1d missing, create them
    if "ret_1d" not in g:
        g["ret_1d"] = g["adj_close"].pct_change()
    if "logret_1d" not in g:
        g["logret_1d"] = np.log(g["adj_close"]).diff()
    lo, hi = g["ret_1d"].quantile([lower_q, upper_q]).values
    g["ret_1d_w"] = g["ret_1d"].clip(lo, hi)
    g["logret_1d_w"] = np.log1p(g["ret_1d_w"])
    return g

panel_w = (panel_aligned
           .groupby("ticker", group_keys=False, sort=False)
           .apply(winsorize_group))

# 3) Rolling features + targets (again ensuring ticker stays)
def add_roll_feats(g):
    t = g.name
    g = g.sort_values("date").copy()
    g["ticker"] = t
    for w in (5, 21, 63):
        g[f"rv_{w}d_w"] = g["logret_1d_w"].rolling(w).std() * np.sqrt(252)
        g[f"ma_{w}d"]   = g["adj_close"].rolling(w).mean()
        g[f"mom_{w}d"]  = g["adj_close"].pct_change(w)
    for h in (5, 21):
        g[f"y_fwdret_{h}d"] = g["adj_close"].pct_change(h).shift(-h)
    return g

panel_w = (panel_w
           .groupby("ticker", group_keys=False, sort=False)
           .apply(add_roll_feats))

need_cols = ["ticker","date","ret_1d_w","logret_1d_w","rv_5d_w","rv_21d_w","rv_63d_w","y_fwdret_5d","y_fwdret_21d"]
panel_w = panel_w.dropna(subset=need_cols).reset_index(drop=True)

print("✅ panel_w rebuilt. Columns:", list(panel_w.columns)[:10], "... (total:", len(panel_w.columns), ")")
print("Has ticker?", "ticker" in panel_w.columns)

# 4) Save tidy + wide
from pathlib import Path
out_dir = Path("data/stock_prices_ready"); out_dir.mkdir(parents=True, exist_ok=True)

panel_w.to_parquet(out_dir / "swiss_panel_ready.parquet", index=False)
panel_w.to_csv(out_dir / "swiss_panel_ready.csv", index=False)

close_wide = (panel_w
              .pivot(index="date", columns="ticker", values="adj_close")
              .sort_index())
close_wide.to_parquet(out_dir / "swiss_close_wide_ready.parquet")
close_wide.to_csv(out_dir / "swiss_close_wide_ready.csv")

print("✅ Saved to:", out_dir)
print("Wide shape:", close_wide.shape)


C:\Users\mkcak\AppData\Local\Temp\ipykernel_30120\211234407.py:38: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(winsorize_group))
C:\Users\mkcak\AppData\Local\Temp\ipykernel_30120\211234407.py:55: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(add_roll_feats))


✅ panel_w rebuilt. Columns: ['date', 'open', 'high', 'low', 'close', 'adj_close', 'volume', 'ticker', 'ret_1d', 'logret_1d'] ... (total: 23 )
Has ticker? True
✅ Saved to: data\stock_prices_ready
Wide shape: (1675, 28)


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

# === 0) Configure paths ===
DATA_DIR = Path(r"C:\Users\mkcak\OneDrive - Hochschule Luzern\Modules\master_thesis\multimodal-finance-forecasting\noteebooks\data\stock_prices_clean")
OUT_DIR  = DATA_DIR.parent / "stock_prices_ready"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# === 1) Load all cleaned CSVs robustly (keeps ticker) ===
def load_prices_clean(data_dir: Path):
    dfs = []
    for p in data_dir.glob("*.csv"):
        df = pd.read_csv(p)
        if df.empty:
            continue

        # Normalize date column name + parse
        if "date" in df.columns:
            df["date"] = pd.to_datetime(df["date"], errors="coerce")
        elif "Date" in df.columns:
            df["date"] = pd.to_datetime(df["Date"], errors="coerce")
        else:
            print(f"⚠️ No date column in {p.name}, skipping")
            continue
        df = df.dropna(subset=["date"])

        # Normalize price columns (handle 'Adj Close' vs 'adj close', etc.)
        cols_lower = {c: c.lower() for c in df.columns}
        df = df.rename(columns=cols_lower)
        if "adj close" in df.columns and "adj_close" not in df.columns:
            df = df.rename(columns={"adj close": "adj_close"})
        if "adj_close" not in df.columns:
            # Fallback to close if adj not present
            if "close" in df.columns:
                df["adj_close"] = df["close"]
            else:
                print(f"⚠️ No adj_close/close in {p.name}, skipping")
                continue

        # Ensure ticker column
        if "ticker" not in df.columns:
            df["ticker"] = p.stem  # use filename

        keep = ["date","open","high","low","close","adj_close","volume","ticker"]
        df = df[[c for c in keep if c in df.columns]]
        dfs.append(df)
    if not dfs:
        raise RuntimeError(f"No usable CSVs found in {data_dir}")
    return pd.concat(dfs, ignore_index=True)

panel = load_prices_clean(DATA_DIR)
panel = panel.sort_values(["ticker","date"]).reset_index(drop=True)
print("Loaded:", panel["ticker"].nunique(), "tickers;", panel.shape, "rows")

# === 2) Align to a common trading calendar (UBSG.SW as reference) ===
REF = "UBSG.SW"
ref_dates = (panel.loc[panel.ticker.eq(REF), ["date"]].drop_duplicates())
panel = panel.merge(ref_dates, on="date", how="inner")
print("After calendar alignment:", panel.shape)

# === 3) Basic returns + winsorization + rolling features + targets ===
def add_features(g):
    g = g.sort_values("date").copy()

    # Simple/log returns
    g["ret_1d"]    = g["adj_close"].pct_change()
    g["logret_1d"] = np.log(g["adj_close"]).diff()

    # Winsorize (per-ticker tails)
    lo, hi = g["ret_1d"].quantile([0.005, 0.995]).values
    g["ret_1d_w"]    = g["ret_1d"].clip(lo, hi)
    g["logret_1d_w"] = np.log1p(g["ret_1d_w"])

    # Rolling features on winsorized log returns + price features
    for w in (5, 21, 63):
        g[f"rv_{w}d_w"] = g["logret_1d_w"].rolling(w).std() * np.sqrt(252)
        g[f"ma_{w}d"]   = g["adj_close"].rolling(w).mean()
        g[f"mom_{w}d"]  = g["adj_close"].pct_change(w)

    # Forward-return targets
    for h in (5, 21):
        g[f"y_fwdret_{h}d"] = g["adj_close"].pct_change(h).shift(-h)
    return g

panel_w = panel.groupby("ticker", group_keys=False).apply(add_features)
need = ["ticker","date","adj_close","ret_1d_w","logret_1d_w","rv_5d_w","rv_21d_w","rv_63d_w","y_fwdret_5d","y_fwdret_21d"]
panel_w = panel_w.dropna(subset=need).reset_index(drop=True)
print("Enriched panel:", panel_w.shape)

# === 4) Save outputs (Parquet + CSV) ===
# Note: requires 'pyarrow' or 'fastparquet' for Parquet I/O
try:
    panel_w.to_parquet(OUT_DIR / "swiss_panel_ready.parquet", index=False)
except Exception as e:
    print("Parquet save skipped (engine missing):", e)
panel_w.to_csv(OUT_DIR / "swiss_panel_ready.csv", index=False)

close_wide = panel_w.pivot(index="date", columns="ticker", values="adj_close").sort_index()
try:
    close_wide.to_parquet(OUT_DIR / "swiss_close_wide_ready.parquet")
except Exception as e:
    print("Parquet save skipped (engine missing):", e)
close_wide.to_csv(OUT_DIR / "swiss_close_wide_ready.csv")

print("✅ Saved to:", OUT_DIR)
print("Wide shape:", close_wide.shape)


Loaded: 28 tickers; (48936, 8) rows
After calendar alignment: (48936, 8)


C:\Users\mkcak\AppData\Local\Temp\ipykernel_30120\2058132570.py:86: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  panel_w = panel.groupby("ticker", group_keys=False).apply(add_features)


Enriched panel: (46584, 23)
✅ Saved to: C:\Users\mkcak\OneDrive - Hochschule Luzern\Modules\master_thesis\multimodal-finance-forecasting\noteebooks\data\stock_prices_ready
Wide shape: (1675, 28)


: 